In [1]:
import pandas as pd

# Load the dataset into a DataFrame
df = pd.read_csv('Data/AB_NYC_2019.csv')

# Rename columns for better clarity
df.rename(columns={'name': 'description', 'id': 'property_id'}, inplace=True)

# Remove rows where all values are missing
df.dropna(how='all', inplace=True)

# Fill missing values for specific columns with default values
df.fillna({'reviews_per_month': 'Unknown', 'last_review': 'Unknown'}, inplace=True)

# Update 'reviews_per_month' to 0 for listings with 0 reviews
df.loc[df['number_of_reviews'] == 0, 'reviews_per_month'] = 0

# Set 'last_review' to 'Never' for listings with 0 reviews
df.loc[df['number_of_reviews'] == 0, 'last_review'] = 'Never'

In [15]:
# Create a new DataFrame containing property-related details
df_property = df[['property_id', 'description', 'neighbourhood', 
                  'room_type', 'price', 'minimum_nights',
                  'reviews_per_month', 'availability_365', 'host_id']]

# Create a separate DataFrame containing host-related details
df_host = df[['host_id', 'host_name', 'calculated_host_listings_count']]

In [17]:
df_property

,property_id,description,neighbourhood,room_type,price,minimum_nights,reviews_per_month,availability_365,host_id
0,2539,Clean & quiet apt home by the park,Kensington,Private room,149,1,0.21,365,2787
1,2595,Skylit Midtown Castle,Midtown,Entire home/apt,225,1,0.38,355,2845
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,Harlem,Private room,150,3,0,365,4632
3,3831,Cozy Entire Floor of Brownstone,Clinton Hill,Entire home/apt,89,1,4.64,194,4869
4,5022,Entire Apt: Spacious Studio/Loft by central park,East Harlem,Entire home/apt,80,10,0.1,0,7192
...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,Bedford-Stuyvesant,Private room,70,2,0,9,8232441
48891,36485057,Affordable room in Bushwick/East Williamsburg,Bushwick,Private room,40,4,0,36,6570630
48892,36485431,Sunny Studio at Historical Neighborhood,Harlem,Entire home/apt,115,10,0,27,23492952
48893,36485609,43rd St. Time Square-cozy single bed,Hell's Kitchen,Shared room,55,1,0,2,30985759


In [19]:
df_host

,host_id,host_name,calculated_host_listings_count
0,2787,John,6
1,2845,Jennifer,2
2,4632,Elisabeth,1
3,4869,LisaRoxanne,1
4,7192,Laura,1
...,...,...,...
48890,8232441,Sabrina,2
48891,6570630,Marisol,2
48892,23492952,Ilgar & Aysel,1
48893,30985759,Taz,6
